## Import modules


In [1]:
from spezi_data_pipeline.data_access.firebase_fhir_data_access import FirebaseFHIRAccess, FHIRResourceType
from spezi_data_pipeline.data_flattening.fhir_resources_flattener import flatten_fhir_resources, FHIRDataFrame, QuestionnaireResponseFlattener
from spezi_data_pipeline.data_processing.data_processor import FHIRDataProcessor
from spezi_data_pipeline.data_processing.questionnaire_processor import calculate_risk_score
from spezi_data_pipeline.data_exploration.data_explorer import DataExplorer, visualizer_factory, explore_total_records_number
from spezi_data_pipeline.data_export.data_exporter import DataExporter

## Define credential files

In [4]:
# Define your Firebase project ID
project_id = "spezi-data-pipeline"  # Replace with your Firebase project ID
project_id = "stanfordspezitemplateapp"

# Define the service account key file
service_account_key_file = "path_to_service_account_key_file.json"  # Replace with your service account key file
service_account_key_file = "stanfordspezitemplateapp-firebase-adminsdk-thsmt-ff5e56378d.json"

# Define the collection name where your FHIR observations are stored and the input code if filtering is needed
collection_name = "users"
subcollection_name = "QuestionnaireResponse"

# Define the survey path(s) for creating survey mappings
survey_path = "Resources/SocialSupportQuestionnaire.json"            

## Initialize the FirebaseFHIRAccess class using your Firebase credentials

In [5]:
# Initialize the FirebaseFHIRAccess class using your Firebase credentials
firebase_access = FirebaseFHIRAccess(project_id, service_account_key_file)
firebase_access.connect()

fhir_questionnaires = firebase_access.fetch_data(collection_name, subcollection_name)

flattened_fhir_dataframe = flatten_fhir_resources(fhir_questionnaires, survey_path)

Unknown Questionnaire Type


In [6]:
flattened_fhir_dataframe.df.head()

,UserId,ResourceId,AuthoredDate,SurveyTitle,QuestionId,QuestionText,AnswerCode,AnswerText
0,2VFMFZpWmpNUGUJrDIa7gooJRD12,37DD265A-B9E8-45FA-87E2-50A132AAEA20,2024-03-27 17:00:24.568549-07:00,Social Support,dcea2683-9815-4505-b240-e75b502b29ef,How often do you need someone to help you if y...,4,Most of the time
1,2VFMFZpWmpNUGUJrDIa7gooJRD12,37DD265A-B9E8-45FA-87E2-50A132AAEA20,2024-03-27 17:00:24.568549-07:00,Social Support,ce09d701-7b93-4150-defb-51825e05ade9,How often do you need someone to take you to t...,4,Most of the time
2,2VFMFZpWmpNUGUJrDIa7gooJRD12,37DD265A-B9E8-45FA-87E2-50A132AAEA20,2024-03-27 17:00:24.568549-07:00,Social Support,58e97564-5f4d-4d4b-86d5-6429cbbc7a8e,How often do you need someone to prepare your ...,4,Most of the time
3,2VFMFZpWmpNUGUJrDIa7gooJRD12,37DD265A-B9E8-45FA-87E2-50A132AAEA20,2024-03-27 17:00:24.568549-07:00,Social Support,ad161c49-e8a6-4d31-90e8-02b2887a765f,How often do you need someone to help with dai...,3,Some of the time
4,2VFMFZpWmpNUGUJrDIa7gooJRD12,37DD265A-B9E8-45FA-87E2-50A132AAEA20,2024-03-27 17:00:24.568549-07:00,Social Support,ba518851-2843-4bbd-c0f7-5b5692d542e0,What is your age?,65,65


## Explore Data

## Export data

In [ ]:
exporter = DataExporter(flattened_fhir_dataframe)
exporter.export_to_csv("survey_data.csv") 

# PHQ-9 example

In [2]:
import json
from fhir.resources.bundle import Bundle
from fhir.resources.questionnaireresponse import QuestionnaireResponse

def load_bundle_from_json(file_path: str) -> Bundle:
    with open(file_path, 'r', encoding='utf-8') as f:
        json_content = json.load(f)
    try:
        bundle = Bundle.parse_obj(json_content)
    except FHIRValidationError as e:
        print(f"Failed to parse Bundle: {e}")
        bundle = None
    return bundle

def extract_questionnaire_responses(bundle: Bundle) -> list:
    questionnaire_responses = []
    for entry in bundle.entry:
        if entry.resource.resource_type == "QuestionnaireResponse":
            try:
                qr = QuestionnaireResponse.parse_obj(entry.resource.dict())
                questionnaire_responses.append(qr)
            except FHIRValidationError as e:
                print(f"Failed to parse QuestionnaireResponse: {e}")
    return questionnaire_responses

file_path = 'sample_data/phq9responses.json'

bundle = load_bundle_from_json(file_path)

if bundle:
    questionnaire_responses = extract_questionnaire_responses(bundle)
    survey_path = 'Resources/PHQ-9.json'
    flattener = QuestionnaireResponseFlattener()
    
    flattened_fhir_dataframe = flattener.flatten(questionnaire_responses, survey_path)

flattened_fhir_dataframe.df.tail()

,UserId,ResourceId,AuthoredDate,SurveyTitle,QuestionId,QuestionText,AnswerCode,AnswerText
40,None,phq9-5,2023-02-26,Patient Health Questionnaire - 9 Item,PHQ9.5,Poor appetite or overeating?,1,Several days
41,None,phq9-5,2023-02-26,Patient Health Questionnaire - 9 Item,PHQ9.6,Feeling bad about yourself - or that you are a...,1,Several days
42,None,phq9-5,2023-02-26,Patient Health Questionnaire - 9 Item,PHQ9.7,"Trouble concentrating on things, such as readi...",0,Not at all
43,None,phq9-5,2023-02-26,Patient Health Questionnaire - 9 Item,PHQ9.8,Moving or speaking so slowly that other people...,1,Several days
44,None,phq9-5,2023-02-26,Patient Health Questionnaire - 9 Item,PHQ9.9,"Thoughts that you would be better off dead, or...",1,Several days


## Calculate risk score

In [3]:
flattened_fhir_dataframe_with_score = calculate_risk_score(flattened_fhir_dataframe)
flattened_fhir_dataframe_with_score.df.head()

,AuthoredDate,SurveyTitle,RiskScore,ScoreInterpretation
0,2023-01-01,Patient Health Questionnaire - 9 Item,24,Severe depression
1,2023-01-15,Patient Health Questionnaire - 9 Item,19,Moderately severe depression
2,2023-01-29,Patient Health Questionnaire - 9 Item,14,Moderate depression
3,2023-02-12,Patient Health Questionnaire - 9 Item,10,Moderate depression
4,2023-02-26,Patient Health Questionnaire - 9 Item,7,Mild depression
